# Project Overview: Australian Economic & Industry Trends Dashboard

This project builds an interactive business intelligence dashboard to monitor Australia’s economic performance through key macroeconomic indicators. It targets analysts, policymakers, and decision-makers who need a clear, data-driven view of economic conditions and trends.

The project follows the **CRISP-DM framework** to ensure a structured data science approach, while also applying dashboard storytelling and semantic modeling best practices to deliver actionable insights.

The final product is a public dashboard hosted on Maven Analytics, supported by a transparent data pipeline built in Google Colab.


#1. Business Understanding:

Explore key macroeconomic indicators (GDP, CPI, unemployment, etc.) to monitor Australia’s economic health and inform public, financial, and policy decisions.

#2. Data Understanding:

## Data Sources Overview

This table lists all selected economic indicators with their official sources and direct download links.

| **Metric**                                  | 🏛️ **Source** | 🔗 **Table / Download Link** |
|--------------------------------------------|---------------|------------------------------|
| **Gross Domestic Product (GDP)**           | ABS           | [Table 2 – Expenditure on GDP (Chain Volume)](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-state-accounts/2023-24-financial-year#data-downloads) |
| **GDP per Capita**                         | ABS           | [Table 2 – Expenditure on GDP (Chain Volume)](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-state-accounts/2023-24-financial-year#data-downloads) |
| **Exports of Goods and Services**          | ABS           | [Table 2 – Expenditure on GDP (Chain Volume)](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-state-accounts/2023-24-financial-year#data-downloads) |
| **Imports of Goods and Services**          | ABS           | [Table 2 – Expenditure on GDP (Chain Volume)](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-state-accounts/2023-24-financial-year#data-downloads) |
| **Consumer Price Index (CPI - Inflation)** | ABS           | [Table 6401.0 – Consumer Price Index](https://www.abs.gov.au/statistics/economy/price-indexes-and-inflation/consumer-price-index-australia/latest-release#data-downloads) |
| **Unemployment Rate**                      | ABS           | [Table 6202.0 – Labour Force, Australia](https://www.abs.gov.au/statistics/labour/employment-and-unemployment/labour-force-australia/latest-release#data-downloads) |
| **Cash Rate (Interest Rate)**              | RBA           | [F1 – Official Cash Rate Target (XLS)](https://www.rba.gov.au/statistics/tables/xls/f01hist.xls) / [Web View](https://www.rba.gov.au/statistics/cash-rate) |
| **Gross Value Added by Industry (GVA)**    | ABS           | [Table 6 – GVA by Industry (Chain Volume)](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-national-income-expenditure-and-product/latest-release#data-downloads) |
| **GDP by State (State Accounts)**          | ABS           | [State Accounts Main Page](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-state-accounts) |


# 3. Data Cleaning and Preparation:

1, Australian Economy Overall

Data loading and DATA EXAMINATION

In [2]:
import pandas as pd

# Load the Excel file
df_gdp_overall = pd.read_excel("/content/Raw GDP Overall.xlsx")

# Display the first few rows of the DataFrame
print("First 5 rows of the DataFrame:")
display(df_gdp_overall.head())

# Display the column names and their data types
print("\nColumn information:")
display(df_gdp_overall.info())

FileNotFoundError: [Errno 2] No such file or directory: '/content/Raw GDP Overall.xlsx'

NORMALIZATION - fact overview table

In [ ]:
# Remove the first 10 rows (0-9) which contain metadata from df_gdp_overall
df_gdp_fact_data = df_gdp_overall.iloc[10:].copy()

# The date information is in the first column ('Unnamed: 0')
date_column = df_gdp_fact_data['Unnamed: 0']

# Select the indicator columns (all columns except the first one)
indicator_columns = df_gdp_fact_data.columns[1:]

# Melt the DataFrame to long format, using the date column as id_vars
df_gdp_fact_normalized = df_gdp_fact_data.melt(
    id_vars=['Unnamed: 0'],
    value_vars=indicator_columns,
    var_name='IndicatorID', # Use the original column name as IndicatorID for now
    value_name='Value'
)

# Rename the date column
df_gdp_fact_normalized = df_gdp_fact_normalized.rename(columns={'Unnamed: 0': 'Date'})

# Display the first few rows of the normalized GDP fact table
print("\nFirst 5 rows of the normalized GDP fact table:")
display(df_gdp_fact_normalized.head())

# Display the column names and their data types of the normalized GDP fact table
print("\nColumn information of the normalized GDP fact table:")
display(df_gdp_fact_normalized.info())

Data Preparation

In [ ]:
# Recheck data types
print("Data types before cleaning:")
display(df_gdp_fact_normalized.dtypes)

# Convert 'Date' column to datetime objects
df_gdp_fact_normalized['Date'] = pd.to_datetime(df_gdp_fact_normalized['Date'], errors='coerce')

# Convert 'Value' column to numeric (float), coercing errors will turn invalid parsing into NaN
df_gdp_fact_normalized['Value'] = pd.to_numeric(df_gdp_fact_normalized['Value'], errors='coerce')

# Ensure 'IndicatorID' is of string type
df_gdp_fact_normalized['IndicatorID'] = df_gdp_fact_normalized['IndicatorID'].astype(str)


# Recheck data types after conversion
print("\nData types after cleaning:")
display(df_gdp_fact_normalized.dtypes)

# Check for null values in the 'Value' column
print("\nNumber of null values in the 'Value' column:")
display(df_gdp_fact_normalized['Value'].isnull().sum())

# Display the first few rows to see the cleaned data
print("\nFirst 5 rows of the cleaned DataFrame:")
display(df_gdp_fact_normalized.head())

Add id

In [ ]:
# Reset df_gdp_fact_normalized to its state after normalization (from cell 2fde6ba6 output)
# This assumes the output of cell 2fde6ba6 is the correct starting point.
# If the notebook state is not as expected, this might need adjustment.

# Re-perform the normalization steps to get the correct starting DataFrame
df_gdp_fact_data = df_gdp_overall.iloc[10:].copy()
indicator_columns = df_gdp_fact_data.columns[1:]
df_gdp_fact_normalized = df_gdp_fact_data.melt(
    id_vars=['Unnamed: 0'],
    value_vars=indicator_columns,
    var_name='IndicatorName_original', # Use a temporary name for original indicator names
    value_name='Value'
)
df_gdp_fact_normalized = df_gdp_fact_normalized.rename(columns={'Unnamed: 0': 'Date'})

# Create a mapping from unique Indicator Names to numerical IDs
unique_indicator_names = df_gdp_fact_normalized['IndicatorName_original'].unique()
indicator_id_map = {name: i + 1 for i, name in enumerate(unique_indicator_names)}

# Add the new 'IndicatorID' column based on the mapping
df_gdp_fact_normalized['IndicatorID'] = df_gdp_fact_normalized['IndicatorName_original'].map(indicator_id_map)

# Rename the original indicator name column to 'Indicator Name' and remove semicolons
df_gdp_fact_normalized = df_gdp_fact_normalized.rename(columns={'IndicatorName_original': 'Indicator Name'})
df_gdp_fact_normalized['Indicator Name'] = df_gdp_fact_normalized['Indicator Name'].str.replace(';', '', regex=False).str.strip()


# Reorder columns to have Date, IndicatorID, Indicator Name, Value
df_gdp_fact_normalized = df_gdp_fact_normalized[['Date', 'IndicatorID', 'Indicator Name', 'Value']]

# Display the first few rows to see the changes
print("First 5 rows of the DataFrame after adding numerical IndicatorID, renaming and cleaning Indicator Name:")
display(df_gdp_fact_normalized.head())

# Display the column names and their data types to confirm changes
print("\nColumn information after changes:")
display(df_gdp_fact_normalized.info())

Create DIM TABLE - dim on indicator info table

In [ ]:
# Extract the first 10 rows as metadata
df_metadata = df_gdp_overall.iloc[:10].copy()

# Transpose the metadata DataFrame
df_metadata_transposed = df_metadata.transpose().reset_index(drop=True)

# Rename the first row as column headers
df_metadata_transposed.columns = df_metadata_transposed.iloc[0]
df_metadata_transposed = df_metadata_transposed[1:].copy()

# Rename the first column to 'Indicator Name' and clean up semicolons and spaces
df_metadata_transposed = df_metadata_transposed.rename(columns={df_metadata_transposed.columns[0]: 'Indicator Name'})
df_metadata_transposed['Indicator Name'] = df_metadata_transposed['Indicator Name'].str.replace(';', '', regex=False).str.strip()

# Create a mapping from cleaned Indicator Names to numerical IDs
unique_indicator_names_cleaned = df_metadata_transposed['Indicator Name'].unique()
indicator_name_to_id_map = {name: i + 1 for i, name in enumerate(unique_indicator_names_cleaned)}

# Add the 'IndicatorID' column to the transposed metadata DataFrame
df_metadata_transposed['IndicatorID'] = df_metadata_transposed['Indicator Name'].map(indicator_name_to_id_map)

# Print the columns of the transposed metadata to identify the correct column names
print("Columns available in transposed metadata:")
print(df_metadata_transposed.columns)

# Define the dimension table with the correct column names
# Based on the print output above, correct the column names if necessary.
# The column that was expected to be 'Unit' appears to be named 'Indicator Name' in the transposed dataframe, and other columns are as expected.
dim_table = df_metadata_transposed[['IndicatorID', 'Indicator Name', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'Series Start', 'Series End', 'No. Obs']].copy()

# Display the first few rows of the dimension table
print("\nFirst 5 rows of the dimension table:")
display(dim_table.head())

# Display the column names and their data types of the dimension table
print("\nColumn information of the dimension table:")
display(dim_table.info())

Exports as csv

In [ ]:
# Export the fact table to a CSV file
df_gdp_fact_normalized.to_csv('gdp_fact_table.csv', index=False)
print("Fact table exported to 'gdp_fact_table.csv'")

# Export the dimension table to a CSV file
dim_table.to_csv('gdp_dim_table.csv', index=False)
print("Dimension table exported to 'gdp_dim_table.csv'")